# First usage of beautifulsoup

In [ ]:
!pip install requests beautifulsoup4 lxml pandas


In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

URL = "https://en.wikipedia.org/wiki/2024%E2%80%9325_Serie_A"
headers = {"User-Agent": "Mozilla/5.0 (educational-webscraping-lab)"}

#Download page
response = requests.get(URL, headers=headers)
response.raise_for_status()
soup = BeautifulSoup(response.text, "lxml")


In [2]:
soup

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>2024–25 Serie A - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-fe

In [8]:
# find all H2 tags and find the one contains "stadium"
target_header = None
for h2 in soup.find_all("h2"):
    if "Stadiums" in h2.get_text():
        target_header = h2
        break

#find the related tables
table = target_header.find_next("table", 
    class_="wikitable")


AttributeError: 'NoneType' object has no attribute 'find_next'

In [4]:
# find all H2 tags and find the one contains "stadium"
target_header = None
for h2 in soup.find_all("h3"):
    if "Stadiums" in h2.get_text():
        target_header = h2
        break

#find the related tables
table = target_header.find_next("table", 
    class_="wikitable")


In [9]:
table

<table class="wikitable sortable">
<tbody><tr>
<th>Team
</th>
<th>Location
</th>
<th>Stadium
</th>
<th>Capacity
</th></tr>
<tr>
<td><a href="/wiki/Atalanta_BC" title="Atalanta BC">Atalanta</a>
</td>
<td><a href="/wiki/Bergamo" title="Bergamo">Bergamo</a>
</td>
<td><a class="mw-redirect" href="/wiki/Gewiss_Stadium" title="Gewiss Stadium">Gewiss Stadium</a>
</td>
<td style="text-align:center;"><span data-sort-value="7004234390000000000♠">23,439</span>
</td></tr>
<tr>
<td><a href="/wiki/Bologna_FC_1909" title="Bologna FC 1909">Bologna</a>
</td>
<td><a href="/wiki/Bologna" title="Bologna">Bologna</a>
</td>
<td><a href="/wiki/Stadio_Renato_Dall%27Ara" title="Stadio Renato Dall'Ara">Stadio Renato Dall'Ara</a>
</td>
<td style="text-align:center;"><span data-sort-value="7004360000000000000♠">36,000</span>
</td></tr>
<tr>
<td><a href="/wiki/Cagliari_Calcio" title="Cagliari Calcio">Cagliari</a>
</td>
<td><a href="/wiki/Cagliari" title="Cagliari">Cagliari</a>
</td>
<td><a href="/wiki/Unipol_Domus

In [14]:
#extract data
rows = []
for tr in table.find_all("tr")[1:]:  #skipe first row
    cells = [td.get_text(strip=True) for td in tr.find_all("td")]
    if len(cells) >= 4:
        rows.append({
            "Team": cells[0],
            "Location": cells[1],
            "Stadium": cells[2],
            "Capacity": cells[3]
        })

df = pd.DataFrame(rows)
df


Team  Location                                         Stadium  \
0        Atalanta   Bergamo                                  Gewiss Stadium   
1         Bologna   Bologna                          Stadio Renato Dall'Ara   
2        Cagliari  Cagliari                                    Unipol Domus   
3            Como      Como                      Stadio Giuseppe Sinigaglia   
4          Empoli    Empoli  Stadio Carlo Castellani - Computer Gross Arena   
5      Fiorentina  Florence                          Stadio Artemio Franchi   
6           Genoa     Genoa                           Stadio Luigi Ferraris   
7   Hellas Verona    Verona                    Stadio Marcantonio Bentegodi   
8     Inter Milan     Milan                                        San Siro   
9        Juventus     Turin                                Juventus Stadium   
10          Lazio      Rome                                 Stadio Olimpico   
11          Lecce     Lecce                             Stadio Via del mare   
12       AC Milan     Milan                                        San Siro   
13          Monza     Monza                                 Stadio Brianteo   
14         Napoli    Naples                   Stadio Diego Armando Maradona   
15          Parma     Parma                            Stadio Ennio Tardini   
16           Roma      Rome                                 Stadio Olimpico   
17         Torino     Turin                   Stadio Olimpico Grande Torino   
18        Udinese     Udine                                   Stadio Friuli   
19        Venezia    Venice                         Stadio Pier Luigi Penzo   

   Capacity  
0    23,439  
1    36,000  
2    16,416  
3    13,602  
4    16,167  
5    43,118  
6    33,205  
7    31,713  
8    75,710  
9    41,507  
10   67,585  
11   30,354  
12   75,710  
13   17,102  
14   54,732  
15   22,352  
16   67,585  
17   28,177  
18   25,132  
19   11,150

In [13]:
from pandas.io import html
from io import StringIO

tables = soup.find_all("table", class_="wikitable")

ranking_html = str(tables[1])


#wrap the html section into an StringIO object
html_buffer = StringIO(ranking_html)

#read table
df1 = html.read_html(html_buffer, flavor="lxml")[0]
df1


Team  Location                                         Stadium  \
0        Atalanta   Bergamo                                  Gewiss Stadium   
1         Bologna   Bologna                          Stadio Renato Dall'Ara   
2        Cagliari  Cagliari                                    Unipol Domus   
3            Como      Como                      Stadio Giuseppe Sinigaglia   
4          Empoli    Empoli  Stadio Carlo Castellani - Computer Gross Arena   
5      Fiorentina  Florence                          Stadio Artemio Franchi   
6           Genoa     Genoa                           Stadio Luigi Ferraris   
7   Hellas Verona    Verona                    Stadio Marcantonio Bentegodi   
8     Inter Milan     Milan                                        San Siro   
9        Juventus     Turin                                Juventus Stadium   
10          Lazio      Rome                                 Stadio Olimpico   
11          Lecce     Lecce                             Stadio Via del mare   
12       AC Milan     Milan                                        San Siro   
13          Monza     Monza                                 Stadio Brianteo   
14         Napoli    Naples                   Stadio Diego Armando Maradona   
15          Parma     Parma                            Stadio Ennio Tardini   
16           Roma      Rome                                 Stadio Olimpico   
17         Torino     Turin                   Stadio Olimpico Grande Torino   
18        Udinese     Udine                                   Stadio Friuli   
19        Venezia    Venice                         Stadio Pier Luigi Penzo   

    Capacity  
0      23439  
1      36000  
2      16416  
3      13602  
4      16167  
5      43118  
6      33205  
7      31713  
8      75710  
9      41507  
10     67585  
11     30354  
12     75710  
13     17102  
14     54732  
15     22352  
16     67585  
17     28177  
18     25132  
19     11150